In [1]:
import json
import numpy as np
import time
from gridfs import GridFS
from pymongo import MongoClient
from PIL import Image

from padma.models.peso.peso2 import N_BINS, PesoModel2

from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.utils.images import generate_batch, get_imagens_recortadas

db = MongoClient(host=MONGODB_URI)[DATABASE]
fs = GridFS(db)

pesomodel = PesoModel2()
print(pesomodel.input_shape)

2019-03-24 15:57:59,275 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/access.log
[15]


/home/ivan/pybr/ajna/padma/padma-venv/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
batch_size = 128
limit = 1280
TEMBBOX = {'metadata.contentType': 'image/jpeg',
          'metadata.predictions.bbox': {'$exists': True}}
dict_filtro = TEMBBOX
projection = ['metadata.predictions']
# dict_filtro['metadata.predictions.index'] = False

In [3]:
db['fs.files'].count_documents(dict_filtro)

33879

In [4]:
batch_gen = generate_batch(db, filtro=dict_filtro, projection=projection,
                           batch_size=batch_size, limit=limit)
model = pesomodel
model.model.n_jobs = 1

In [5]:
X = np.zeros((batch_size, *model.input_shape), dtype=np.float32)
_ids = []
y = []
s = time.time()
for batch, rows in batch_gen:
    if len(batch) == 0:
        break
    s0 = time.time()
    for i, (images, row) in enumerate(zip(batch, rows)):
        x = model.prepara(images[0])
        X[i, :] = x
        _ids.append(row['_id'])
    s1 = time.time()
    print('Montou X em %0.2f ' % (s1 - s0))
    preds = model.model.predict(X)
    y.append(preds)
    s2 = time.time()
    print('Fez predição em %s' % (s2 - s1))
    # print(indexes)
s3 = time.time()
elapsed = s3 - s
print('Tempo total: %s. Por imagem: %s' % (elapsed, elapsed / limit))
    

Montou X em 0.39 
Fez predição em 0.002807140350341797
Montou X em 0.32 
Fez predição em 0.002011537551879883
Montou X em 0.33 
Fez predição em 0.0022270679473876953
Montou X em 0.41 
Fez predição em 0.0022029876708984375
Montou X em 0.36 
Fez predição em 0.0015447139739990234
Montou X em 0.34 
Fez predição em 0.0024306774139404297
Montou X em 0.45 
Fez predição em 0.0017740726470947266
Montou X em 0.29 
Fez predição em 0.0017902851104736328
Montou X em 0.39 
Fez predição em 0.0017571449279785156
Montou X em 0.29 
Fez predição em 0.0015561580657958984
Tempo total: 15.500345468521118. Por imagem: 0.012109644897282123


## Abaixo, testes e visualizações de "sanidade", isto é, as predições são iguais às do PADMA???

In [7]:
preds.shape

(128,)

In [8]:
ystack = np.vstack(y).astype(np.float32).flatten()
ystack.shape

(1280,)

In [9]:
len(_ids)

1280

In [10]:
# TODO: Fazer Update no MongoDB

In [11]:
for i, _id in enumerate(_ids[:10]):
    index_row = db.fs.files.find_one({'_id': _id})
    old_peso = index_row['metadata']['predictions'][0]['peso']
    new_peso = ystack[i]
    print(old_peso, new_peso)
    dif = abs(old_peso - new_peso)
    if dif > 1.:
        print(dif)

28337.480494505493 28108.828
228.65236950549297
6025.895937660854 6025.896
29657.64348176531 29657.643
27224.94695619157 27224.947
23143.433933730157 25767.314
2623.880519394843
13755.541070806541 13755.541
25128.535906166042 24828.082
300.45387491604197
17430.17714098451 16868.428
561.7494066095096
15461.983241610049 15624.591
162.60757870245106
7816.210324935654 7613.662
202.54821556065417


In [12]:
# db.fs.files.update_one({'_id': _ids[0]}, {'$set': {'metadata.predicitions': json.dumps(new_predictions)}})

In [15]:
array = np.asarray(images[0]).flatten() #, dtype=np.float32)
# print(array)
# Verify they give the same answer
H_np = np.histogram(array, bins=16, range=(0, 255))[0]
H_ds = np.histogram(array, bins=16)[0]
# print(H_np)
# print(H_ds)

print ('Same answer:', (H_np == H_ds).all())

# Timeit
%timeit np.histogram(array, bins=16, range=(0, 255))
%timeit np.histogram(array, bins=16)

Same answer: True
1.33 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.84 ms ± 702 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
